## ПРОЕКТ "РЕАКЦИЯ РЫНКОВ НА ВЫСТУПЛЕНИЯ ГЛАВ ЦЕНТРАЛЬНЫХ БАНКОВ"

Банком международных расчетов опубликованы тексты выступлений глав центральных банков разных стран с 1996 года по настоящее время (https://www.bis.org/cbspeeches/download.htm). В рамках проекта поставлена задача подготовить модель, которая бы прогнозировала реакцию рынков (на примере Индекса американских компаний S&P500) на текст выступления глав регуляторов (на примере выступлений представителей ФРС)

In [59]:
#импортируем библиотеки
import pandas as pd
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.ensemble import RandomForestClassifier
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, mean_squared_error, r2_score
#from pymystem3 import Mystem
#m = Mystem()
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('omw-1.4')
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
from nltk.corpus import stopwords
nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords')
import re

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Илья\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Илья\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\Илья\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Илья\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Илья\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
#открываем предоставленные данные, выступления глав ЦБ
speeches = pd.read_csv('C:\\Users\\Илья\\Documents\\Программирование\\Data Science_projects\\BIS\\speeches.csv')

In [62]:
#выступления исключительно 2024 года
speeches_2024 = pd.read_csv('C:\\Users\\Илья\\Documents\\Программирование\\Data Science_projects\\BIS\\speeches_2024.csv')

In [3]:
#динамика индекса американских компаний, S&P500
spx = pd.read_csv('C:\\Users\\Илья\\Documents\\Программирование\\Data Science_projects\\BIS\\SPX.csv')

### Анализ публикаций 2024 года

In [66]:
#смотрим на представленные данные
display(speeches_2024.head())
print()
print(speeches_2024.shape)
print()
print(speeches_2024.dtypes)

,url,title,description,date,text,author
0,https://www.bis.org/review/r240109a.htm,Michelle W Bowman: New year's resolutions for ...,"Speech by Ms Michelle W Bowman, Member of the ...",2024-12-08 00:00:00,For release on delivery\n4:30 p.m. EST\nJanuar...,Michelle W Bowman
1,https://www.bis.org/review/r240123b.htm,Michelle W Bowman: The path forward for bank c...,"Speech (virtual) by Ms Michelle W Bowman, Memb...",2024-01-17 00:00:00,For release on delivery\n9:00 a.m. EST\nJanuar...,Michelle W Bowman
2,https://www.bis.org/review/r240123a.htm,Michael S Barr: Measuring cyber risk in the fi...,"Opening remarks by Mr Michael S Barr, Vice Cha...",2024-01-17 00:00:00,Michael S Barr: Measuring cyber risk in the fi...,Michael S Barr
3,https://www.bis.org/review/r240117a.htm,John C Williams: Rules of three,"Remarks by Mr John C Williams, President and C...",2024-01-10 00:00:00,FEDERAL RESERVE BANK of NEW YORK Serving the S...,John C Williams
4,https://www.bis.org/review/r240117g.htm,Swaminathan J: Resolution of stressed assets a...,"Speech by Mr Swaminathan J, Deputy Governor of...",2024-01-10 00:00:00,Swaminathan J: Resolution of stressed assets a...,Shri Swaminathan J



(28, 6)

url            object
title          object
description    object
date           object
text           object
author         object
dtype: object


- публикации с 10 января по 8 декабря. Декабрьские публикации - явно ошибка

In [63]:
#период публикаций
print(speeches_2024['date'].min())
print(speeches_2024['date'].max())

2024-01-10 00:00:00
2024-12-08 00:00:00


In [64]:
#авторы публикаций
print(speeches_2024['author'].unique())

['Michelle W Bowman' 'Michael S Barr' 'John C Williams'
 'Shri Swaminathan J' 'Shaktikanta Das' 'Luis de Guindos'
 'Christopher J Waller' 'Michael Debabrata Patra'
 'Encik Adnan Zaylani Mohamad Zahid' 'Tiff Macklem' 'Fabio Panetta'
 'Ida Wolden Bache' 'Luigi Federico Signorini' 'Piero Cipollone'
 'Gabriel Makhlouf' 'Boris Vujčić' 'Jorgovanka Tabaković' 'Dimitar Radev'
 'Burkhard Balz' 'Andriy Pyshnyy']


In [65]:
#уникальные описания
print(speeches_2024['description'].unique())

['Speech by Ms Michelle W Bowman, Member of the Board of Governors of the Federal Reserve System, at the South Carolina Bankers Association 2024 Community Bankers Conference, Columbia, South Carolina, 8 December 2024.'
 'Speech (virtual) by Ms Michelle W Bowman, Member of the Board of Governors of the Federal Reserve System, at the Protect Main Street, sponsored by the Center for Capital Markets at the US Chamber of Commerce, Washington DC, 17 January 2024.'
 'Opening remarks by Mr Michael S Barr, Vice Chair for Supervision of the Board of Governors of the Federal Reserve System, at the conference on Measuring Cyber Risk in the Financial Services Sector, Boston, Massachusetts, 17 January 2024.'
 "Remarks by Mr John C Williams, President and Chief Executive Officer of the Federal Reserve Bank of New York, at the Bronx EDC and BICNY's 2024 Regional Economic Outlook, White Plains, New York, 10 January 2024. "
 'Speech by Mr Swaminathan J, Deputy Governor of the Reserve Bank of India, at t

Пропуски отсутствуют

In [67]:
#проверка на пропуски
print(speeches_2024.isna().sum())

url            0
title          0
description    0
date           0
text           0
author         0
dtype: int64


Создали справочник с должностью и организацией выступающих

In [68]:
#справочник
col = ['position', 'organization']
df_data = [['Member of the Board of Governors of the Federal Reserve System', 'Federal Reserve System'],
['Member of the Board of Governors of the Federal Reserve System', 'Federal Reserve System'],
['Vice Chair for Supervision of the Board of Governors of the Federal Reserve System', 'Federal Reserve System'],
['President and Chief Executive Officer of the Federal Reserve Bank of New York', 'Federal Reserve Bank of New York'],
['Deputy Governor of the Reserve Bank of India', 'Reserve Bank of India'],
['Deputy Governor of the Reserve Bank of India', 'Reserve Bank of India'],
['Governor of the Reserve Bank of India', 'Reserve Bank of India'],
['Governor of the Reserve Bank of India', 'Reserve Bank of India'],
['Vice-President of the European Central Bank', 'European Central Bank'],
['Member of the Board of Governors of the Federal Reserve System', 'Federal Reserve System'],
['Deputy Governor of the Reserve Bank of India', 'Reserve Bank of India'],
['Deputy Governor of the Reserve Bank of India', 'Reserve Bank of India'],
['Deputy Governor of the Reserve Bank of India', 'Reserve Bank of India'],
['Governor of the Reserve Bank of India', 'Reserve Bank of India'],
['Deputy Governor of the Central Bank of Malaysia', 'Central Bank of Malaysia'],
['Governor of the Bank of Canada', 'Bank of Canada'],
['Governor of the Bank of Italy', 'Bank of Italy'],
['Governor of Norges Bank (Central Bank of Norway)', 'Central Bank of Norway'],
['Senior Deputy Governor of the Bank of Italy', 'Bank of Italy'],
['Deputy Governor of the Bank of Italy', 'Bank of Italy'],
['Governor of the Central Bank of Ireland', 'Central Bank of Ireland'],
['Governor of the Croatian National Bank', 'Croatian National Bank'],
['Member of the Board of Governors of the Federal Reserve System', 'Federal Reserve System'],
['Governor of the National Bank of Serbia', 'National Bank of Serbia'],
['Governor of the Bulgarian National Bank', 'Bulgarian National Bank'],
['Member of the Executive Board of the Deutsche Bundesbank', 'Deutsche Bundesbank'],
['Governor of the Bank of Canada', 'Bank of Canada'],
['Governor of the National Bank of Ukraine', 'National Bank of Ukraine']]
df_pos_org = pd.DataFrame(columns = col, data = df_data)

In [69]:
#смотрим на представленные данные
display(df_pos_org.head())
print()
print(df_pos_org.shape)

,position,organization
0,Member of the Board of Governors of the Federa...,Federal Reserve System
1,Member of the Board of Governors of the Federa...,Federal Reserve System
2,Vice Chair for Supervision of the Board of Gov...,Federal Reserve System
3,President and Chief Executive Officer of the F...,Federal Reserve Bank of New York
4,Deputy Governor of the Reserve Bank of India,Reserve Bank of India



(28, 2)


Присоединили справочник к основной таблице

In [70]:
#присоединяем к основной табл
speeches_2024_2 = speeches_2024.merge(df_pos_org, how='left', left_index=True, right_index=True)

In [71]:
display(speeches_2024_2.head())
print()
display(speeches_2024_2.tail())

,url,title,description,date,text,author,position,organization
0,https://www.bis.org/review/r240109a.htm,Michelle W Bowman: New year's resolutions for ...,"Speech by Ms Michelle W Bowman, Member of the ...",2024-12-08 00:00:00,For release on delivery\n4:30 p.m. EST\nJanuar...,Michelle W Bowman,Member of the Board of Governors of the Federa...,Federal Reserve System
1,https://www.bis.org/review/r240123b.htm,Michelle W Bowman: The path forward for bank c...,"Speech (virtual) by Ms Michelle W Bowman, Memb...",2024-01-17 00:00:00,For release on delivery\n9:00 a.m. EST\nJanuar...,Michelle W Bowman,Member of the Board of Governors of the Federa...,Federal Reserve System
2,https://www.bis.org/review/r240123a.htm,Michael S Barr: Measuring cyber risk in the fi...,"Opening remarks by Mr Michael S Barr, Vice Cha...",2024-01-17 00:00:00,Michael S Barr: Measuring cyber risk in the fi...,Michael S Barr,Vice Chair for Supervision of the Board of Gov...,Federal Reserve System
3,https://www.bis.org/review/r240117a.htm,John C Williams: Rules of three,"Remarks by Mr John C Williams, President and C...",2024-01-10 00:00:00,FEDERAL RESERVE BANK of NEW YORK Serving the S...,John C Williams,President and Chief Executive Officer of the F...,Federal Reserve Bank of New York
4,https://www.bis.org/review/r240117g.htm,Swaminathan J: Resolution of stressed assets a...,"Speech by Mr Swaminathan J, Deputy Governor of...",2024-01-10 00:00:00,Swaminathan J: Resolution of stressed assets a...,Shri Swaminathan J,Deputy Governor of the Reserve Bank of India,Reserve Bank of India


,url,title,description,date,text,author,position,organization
23,https://www.bis.org/review/r240202d.htm,Jorgovanka Tabaković: Transitions big and small,Introductory address by Dr Jorgovanka Tabakovi...,2024-01-31 00:00:00,Jorgovanka Tabakovi: Transitions big and small...,Jorgovanka Tabaković,Governor of the National Bank of Serbia,National Bank of Serbia
24,https://www.bis.org/review/r240202c.htm,Dimitar Radev: 2024 - the focus is still on cu...,"Publication by Mr Dimitar Radev, Governor of t...",2024-02-02 09:44:00,Dimitar Radev: 2024 - the focus is still on cu...,Dimitar Radev,Governor of the Bulgarian National Bank,Bulgarian National Bank
25,https://www.bis.org/review/r240202b.htm,Burkhard Balz: Overcoming challenges together,"Address by Mr Burkhard Balz, Member of the Exe...",2024-01-31 00:00:00,Burkhard Balz: Overcoming challenges together\...,Burkhard Balz,Member of the Executive Board of the Deutsche ...,Deutsche Bundesbank
26,https://www.bis.org/review/r240202a.htm,Tiff Macklem: Opening statement before the Hou...,"Opening statement by Mr Tiff Macklem, Governor...",2024-02-01 00:00:00,Tiff Macklem: Opening statement before the Hou...,Tiff Macklem,Governor of the Bank of Canada,Bank of Canada
27,https://www.bis.org/review/r240202e.htm,Andriy Pyshnyy: National Bank of Ukraine press...,"Speech by Mr Andriy Pyshnyy, Governor of the N...",2024-01-25 00:00:00,Andriy Pyshnyy: National Bank of Ukraine press...,Andriy Pyshnyy,Governor of the National Bank of Ukraine,National Bank of Ukraine


Больше всего выступлений у Reserve Bank of India и Federal Reserve System (США)

In [72]:
display(speeches_2024_2.pivot_table(index='organization', values='title', aggfunc='count'))

,title
organization,
Bank of Canada,2
Bank of Italy,3
Bulgarian National Bank,1
Central Bank of Ireland,1
Central Bank of Malaysia,1
Central Bank of Norway,1
Croatian National Bank,1
Deutsche Bundesbank,1
European Central Bank,1


#### Дательнее посмотрим на выступления представителей FRS

In [73]:
#детальнее посмотрим на Federal Reserve System и Federal Reserve Bank of New York
speeches_2024_frs = speeches_2024_2.query('organization in ["Federal Reserve System", "Federal Reserve Bank of New York"]')
print(speeches_2024_frs.shape)

(6, 8)


В основном по банковской тематике выступали, что логично

In [74]:
#на какие темы выступали представители ФРС
print(speeches_2024_frs['title'].unique())

["Michelle W Bowman: New year's resolutions for bank regulatory policymakers"
 'Michelle W Bowman: The path forward for bank capital reform'
 'Michael S Barr: Measuring cyber risk in the financial services sector'
 'John C Williams: Rules of three'
 'Christopher J Waller: Almost as good as it gets-but will it last?'
 'Michelle W Bowman: The future of banking']


#### Дательнее посмотрим на выступления представителей RBI

In [76]:
#детальнее посмотрим на Reserve Bank of India
speeches_2024_rbi = speeches_2024_2.query('organization == "Reserve Bank of India"').reset_index(drop=True)
display(speeches_2024_rbi.head())
print(speeches_2024_rbi.shape)

,url,title,description,date,text,author,position,organization
0,https://www.bis.org/review/r240117g.htm,Swaminathan J: Resolution of stressed assets a...,"Speech by Mr Swaminathan J, Deputy Governor of...",2024-01-10 00:00:00,Swaminathan J: Resolution of stressed assets a...,Shri Swaminathan J,Deputy Governor of the Reserve Bank of India,Reserve Bank of India
1,https://www.bis.org/review/r240117h.htm,Swaminathan J: Safeguarding financial stabilit...,"Speech by Mr Swaminathan J, Deputy Governor of...",2024-01-10 00:00:00,Swaminathan J: Safeguarding financial stabilit...,Shri Swaminathan J,Deputy Governor of the Reserve Bank of India,Reserve Bank of India
2,https://www.bis.org/review/r240117e.htm,Shaktikanta Das: Turnaround of the Indian bank...,"Keynote address by Mr Shaktikanta Das, Governo...",2024-01-11 00:00:00,Turnaround of the Indian Banking System\n\nKey...,Shaktikanta Das,Governor of the Reserve Bank of India,Reserve Bank of India
3,https://www.bis.org/review/r240117f.htm,Shaktikanta Das: Insolvency and bankruptcy cod...,"Keynote address by Mr Shaktikanta Das, Governo...",2024-01-11 00:00:00,Shaktikanta Das: Insolvency and bankruptcy cod...,Shaktikanta Das,Governor of the Reserve Bank of India,Reserve Bank of India
4,https://www.bis.org/review/r240130h.htm,Michael Debabrata Patra: Harnessing digital te...,"Keynote address by Dr Michael Debabrata Patra,...",2024-01-17 00:00:00,Michael Debabrata Patra: Harnessing digital te...,Michael Debabrata Patra,Deputy Governor of the Reserve Bank of India,Reserve Bank of India


(8, 8)


Также в основном на банковскую тематику

In [77]:
#на какие темя выступали представители Банка Индии
print(speeches_2024_rbi['title'].unique())

['Swaminathan J: Resolution of stressed assets and IBC – the future road map'
 'Swaminathan J: Safeguarding financial stability - the crucial role of assurance functions'
 'Shaktikanta Das: Turnaround of the Indian banking system'
 'Shaktikanta Das: Insolvency and bankruptcy code – towards achieving full potential'
 'Michael Debabrata Patra: Harnessing digital technologies in central banks - opportunities and challenges'
 'Swaminathan J: The vital role of Internal Ombudsman in ensuring customer-centric financial excellence'
 'Swaminathan J: Role and expectations of directors of urban co-operative banks - upholding governance and professionalism with capacity buidling and technology upgradation'
 "Shaktikanta Das: India's journey from crisis to confidence"]


### Анализ публикаций 1996 - 2024 гг.

In [4]:
#смотрим на представленные данные
display(speeches.head())
print()
print(speeches.shape)
print()
print(speeches.dtypes)

,url,title,description,date,text,author
0,https://www.bis.org/review/r971001f.pdf,Mr. Bäckström elucidates the problems Sweden w...,Talk given by the Governor of the Swedish Risk...,1987-08-29 00:00:00,Mr. Bäckström elucidates the problems Sweden...,Urban Bäckström
1,https://www.bis.org/review/r970211c.pdf,Mr. Chen discusses monetary relations between ...,Speech by the Deputy Governor of the People's ...,1996-09-10 00:00:00,Mr. Chen discusses monetary relations between ...,Chen Yuan
2,https://www.bis.org/review/r970211b.pdf,Mr. Dai looks at the possibilities of strength...,Speech by the Governor of the People's Bank of...,1996-11-13 00:00:00,Mr. Dai looks at the possibilities of strength...,Dai Xianglong
3,https://www.bis.org/review/r970211a.pdf,Mr. Dai assesses the outlook for Hong Kong as ...,Speech by the Governor of the People's Bank of...,1996-09-30 00:00:00,Mr. Dai assesses the outlook for Hong Kong as ...,Dai Xianglong
4,https://www.bis.org/review/r970203b.pdf,Mr. Rangarajan examines the objectives of mone...,Address by the Governor of the Reserve Bank of...,1996-12-28 00:00:00,Mr. Rangarajan examines the objectives of mone...,Bimal Jalan



(18827, 6)

url            object
title          object
description    object
date           object
text           object
author         object
dtype: object


Публикации с 1987 по 2024. Публикации от декабря 2024 явно ошибка

In [78]:
#период публикаций
print(speeches['date'].min())
print(speeches['date'].max())

1987-08-29 00:00:00
2024-12-08 00:00:00


Для анализа взяли публикации с 1996 по 2023 гг.

In [5]:
#возьмем публикации с 1996 по 2023 гг.
speeches_1996_2023 = speeches.query('date >= "1996-01-01" and date <= "2023-12-31"')

Убрали пропуск в столбце "description"

In [6]:
#пропуски
print(speeches_1996_2023.isna().sum())

url             0
title           0
description     1
date            0
text            0
author         25
dtype: int64


In [7]:
speeches_1996_2023 = speeches_1996_2023.dropna(subset=['description'])
print(speeches_1996_2023.shape)

(18797, 6)


Для дальнейшей работы взяли выступления исключительно представителей ФРС

In [8]:
#публикации ФРС
speeches_1996_2023_frs = speeches_1996_2023.query('description.str.contains("Federal Reserve")', engine='python')

In [79]:
display(speeches_1996_2023_frs.head())
print()
print(speeches_1996_2023_frs.shape)

,url,title,description,date,text,author,date2,date_datetime
7,https://www.bis.org/review/r970108b.pdf,Ms. Rivlin discusses the prudential regulation...,Remarks by the Vice Chairman of the Board of G...,1996-12-19 00:00:00,Ms. Rivlin discusses the prudential regulation...,Alice M Rivlin,1996-12-19,1996-12-19
11,https://www.bis.org/review/r970512a.pdf,Mr. Meyer discusses the economic outlook and t...,"Remarks by Mr. Laurence H. Meyer, a member of ...",1997-04-24 00:00:00,Mr. Meyer discusses the economic outlook and t...,Laurence H Meyer,1997-04-24,1997-04-24
19,https://www.bis.org/review/r980102e.pdf,Mr. Kelley discusses the 'Millennium Bug' from...,"Remarks by Mr. Edward W. Kelley, Jr., a member...",1997-12-15 00:00:00,"Mr. Kelley discusses the ""Millennium Bug"" from...","Edward W Kelley, Jr",1997-12-15,1997-12-15
29,https://www.bis.org/review/r971208d.pdf,Mr. Greenspan's remarks to the Economic Club o...,Remarks by the Chairman of the Board of the U ...,1997-12-02 00:00:00,Mr. Greenspan's remarks to the Economic Club o...,Alan Greenspan,1997-12-02,1997-12-02
33,https://www.bis.org/review/r971202b.pdf,Mr. McDonough gives a US perspective on Econom...,Remarks by the President of the Federal Reserv...,1997-11-17 00:00:00,Mr. McDonough gives a US perspective on Econom...,William J McDonough,1997-11-17,1997-11-17



(2345, 8)


Доля выступлений ФРС - 12%

In [80]:
print(speeches_1996_2023_frs.shape[0] / speeches_1996_2023.shape[0])

0.12475395009841996


In [9]:
#типы данных
print(speeches_1996_2023_frs.dtypes)

url            object
title          object
description    object
date           object
text           object
author         object
dtype: object


Добавили признак date2 без времени

In [10]:
speeches_1996_2023_frs['date2'] = pd.to_datetime(speeches_1996_2023_frs['date']).dt.date

C:\Users\Илья\AppData\Local\Temp\ipykernel_12108\3646647569.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  speeches_1996_2023_frs['date2'] = pd.to_datetime(speeches_1996_2023_frs['date']).dt.date


In [11]:
print(speeches_1996_2023_frs.dtypes)
print()
display(speeches_1996_2023_frs.head())

url            object
title          object
description    object
date           object
text           object
author         object
date2          object
dtype: object



,url,title,description,date,text,author,date2
7,https://www.bis.org/review/r970108b.pdf,Ms. Rivlin discusses the prudential regulation...,Remarks by the Vice Chairman of the Board of G...,1996-12-19 00:00:00,Ms. Rivlin discusses the prudential regulation...,Alice M Rivlin,1996-12-19
11,https://www.bis.org/review/r970512a.pdf,Mr. Meyer discusses the economic outlook and t...,"Remarks by Mr. Laurence H. Meyer, a member of ...",1997-04-24 00:00:00,Mr. Meyer discusses the economic outlook and t...,Laurence H Meyer,1997-04-24
19,https://www.bis.org/review/r980102e.pdf,Mr. Kelley discusses the 'Millennium Bug' from...,"Remarks by Mr. Edward W. Kelley, Jr., a member...",1997-12-15 00:00:00,"Mr. Kelley discusses the ""Millennium Bug"" from...","Edward W Kelley, Jr",1997-12-15
29,https://www.bis.org/review/r971208d.pdf,Mr. Greenspan's remarks to the Economic Club o...,Remarks by the Chairman of the Board of the U ...,1997-12-02 00:00:00,Mr. Greenspan's remarks to the Economic Club o...,Alan Greenspan,1997-12-02
33,https://www.bis.org/review/r971202b.pdf,Mr. McDonough gives a US perspective on Econom...,Remarks by the President of the Federal Reserv...,1997-11-17 00:00:00,Mr. McDonough gives a US perspective on Econom...,William J McDonough,1997-11-17


Добавили признак date_datetime в формате даты

In [12]:
speeches_1996_2023_frs['date_datetime'] = pd.to_datetime(speeches_1996_2023_frs['date2'], format='%Y-%m-%d')

C:\Users\Илья\AppData\Local\Temp\ipykernel_12108\3354746500.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  speeches_1996_2023_frs['date_datetime'] = pd.to_datetime(speeches_1996_2023_frs['date2'], format='%Y-%m-%d')


In [13]:
print(speeches_1996_2023_frs.dtypes)
print()
display(speeches_1996_2023_frs.head())

url                      object
title                    object
description              object
date                     object
text                     object
author                   object
date2                    object
date_datetime    datetime64[ns]
dtype: object



,url,title,description,date,text,author,date2,date_datetime
7,https://www.bis.org/review/r970108b.pdf,Ms. Rivlin discusses the prudential regulation...,Remarks by the Vice Chairman of the Board of G...,1996-12-19 00:00:00,Ms. Rivlin discusses the prudential regulation...,Alice M Rivlin,1996-12-19,1996-12-19
11,https://www.bis.org/review/r970512a.pdf,Mr. Meyer discusses the economic outlook and t...,"Remarks by Mr. Laurence H. Meyer, a member of ...",1997-04-24 00:00:00,Mr. Meyer discusses the economic outlook and t...,Laurence H Meyer,1997-04-24,1997-04-24
19,https://www.bis.org/review/r980102e.pdf,Mr. Kelley discusses the 'Millennium Bug' from...,"Remarks by Mr. Edward W. Kelley, Jr., a member...",1997-12-15 00:00:00,"Mr. Kelley discusses the ""Millennium Bug"" from...","Edward W Kelley, Jr",1997-12-15,1997-12-15
29,https://www.bis.org/review/r971208d.pdf,Mr. Greenspan's remarks to the Economic Club o...,Remarks by the Chairman of the Board of the U ...,1997-12-02 00:00:00,Mr. Greenspan's remarks to the Economic Club o...,Alan Greenspan,1997-12-02,1997-12-02
33,https://www.bis.org/review/r971202b.pdf,Mr. McDonough gives a US perspective on Econom...,Remarks by the President of the Federal Reserv...,1997-11-17 00:00:00,Mr. McDonough gives a US perspective on Econom...,William J McDonough,1997-11-17,1997-11-17


### Смотрим на данные Индекса S&P500

In [14]:
#анализируем представленные данные
display(spx.head())
print()
print(spx.shape)

,Date,Open,High,Low,Close,Adj Close,Volume
0,1987-01-02,242.169998,246.449997,242.169998,246.449997,246.449997,91880000
1,1987-01-05,246.449997,252.570007,246.449997,252.190002,252.190002,181900000
2,1987-01-06,252.199997,253.990005,252.139999,252.779999,252.779999,189300000
3,1987-01-07,252.779999,255.720001,252.649994,255.330002,255.330002,190900000
4,1987-01-08,255.360001,257.279999,254.970001,257.279999,257.279999,194500000



(9399, 7)


Добавили признак date_datetime в формате даты

In [15]:
spx['date_datetime'] = pd.to_datetime(spx['Date'], format='%Y-%m-%d')

In [16]:
display(spx.head())
print()
print(spx.dtypes)

,Date,Open,High,Low,Close,Adj Close,Volume,date_datetime
0,1987-01-02,242.169998,246.449997,242.169998,246.449997,246.449997,91880000,1987-01-02
1,1987-01-05,246.449997,252.570007,246.449997,252.190002,252.190002,181900000,1987-01-05
2,1987-01-06,252.199997,253.990005,252.139999,252.779999,252.779999,189300000,1987-01-06
3,1987-01-07,252.779999,255.720001,252.649994,255.330002,255.330002,190900000,1987-01-07
4,1987-01-08,255.360001,257.279999,254.970001,257.279999,257.279999,194500000,1987-01-08



Date                     object
Open                    float64
High                    float64
Low                     float64
Close                   float64
Adj Close               float64
Volume                    int64
date_datetime    datetime64[ns]
dtype: object


Оставили только нужные столбцы

In [17]:
#оставляем нужные столбцы
spx_drop = spx.drop(['Date', 'Open', 'High', 'Low', 'Adj Close', 'Volume'], axis=True)
display(spx_drop.head())

,Close,date_datetime
0,246.449997,1987-01-02
1,252.190002,1987-01-05
2,252.779999,1987-01-06
3,255.330002,1987-01-07
4,257.279999,1987-01-08


### Добавили к df с выступлениями значения Индекса S&P500

In [18]:
speeches_1996_2023_frs_merge = speeches_1996_2023_frs.merge(spx_drop, on='date_datetime', how='left')

In [19]:
display(speeches_1996_2023_frs_merge.head())
print()
print(speeches_1996_2023_frs_merge.shape)

,url,title,description,date,text,author,date2,date_datetime,Close
0,https://www.bis.org/review/r970108b.pdf,Ms. Rivlin discusses the prudential regulation...,Remarks by the Vice Chairman of the Board of G...,1996-12-19 00:00:00,Ms. Rivlin discusses the prudential regulation...,Alice M Rivlin,1996-12-19,1996-12-19,745.760010
1,https://www.bis.org/review/r970512a.pdf,Mr. Meyer discusses the economic outlook and t...,"Remarks by Mr. Laurence H. Meyer, a member of ...",1997-04-24 00:00:00,Mr. Meyer discusses the economic outlook and t...,Laurence H Meyer,1997-04-24,1997-04-24,771.179993
2,https://www.bis.org/review/r980102e.pdf,Mr. Kelley discusses the 'Millennium Bug' from...,"Remarks by Mr. Edward W. Kelley, Jr., a member...",1997-12-15 00:00:00,"Mr. Kelley discusses the ""Millennium Bug"" from...","Edward W Kelley, Jr",1997-12-15,1997-12-15,963.390015
3,https://www.bis.org/review/r971208d.pdf,Mr. Greenspan's remarks to the Economic Club o...,Remarks by the Chairman of the Board of the U ...,1997-12-02 00:00:00,Mr. Greenspan's remarks to the Economic Club o...,Alan Greenspan,1997-12-02,1997-12-02,971.679993
4,https://www.bis.org/review/r971202b.pdf,Mr. McDonough gives a US perspective on Econom...,Remarks by the President of the Federal Reserv...,1997-11-17 00:00:00,Mr. McDonough gives a US perspective on Econom...,William J McDonough,1997-11-17,1997-11-17,946.200012



(2345, 9)


Убрали пропуски в столбце "Close"

In [20]:
print(speeches_1996_2023_frs_merge.isna().sum())

url                0
title              0
description        0
date               0
text               0
author             1
date2              0
date_datetime      0
Close            129
dtype: int64


In [21]:
#убираем пропуски
speeches_1996_2023_frs_merge = speeches_1996_2023_frs_merge.dropna(subset=['Close'])

In [22]:
print(speeches_1996_2023_frs_merge.isna().sum())

url              0
title            0
description      0
date             0
text             0
author           1
date2            0
date_datetime    0
Close            0
dtype: int64


Переименовали столбец

In [23]:
#переименовываем столбцы
speeches_1996_2023_frs_merge = speeches_1996_2023_frs_merge.rename(columns={'Close': 'close'})
display(speeches_1996_2023_frs_merge.head())

,url,title,description,date,text,author,date2,date_datetime,close
0,https://www.bis.org/review/r970108b.pdf,Ms. Rivlin discusses the prudential regulation...,Remarks by the Vice Chairman of the Board of G...,1996-12-19 00:00:00,Ms. Rivlin discusses the prudential regulation...,Alice M Rivlin,1996-12-19,1996-12-19,745.760010
1,https://www.bis.org/review/r970512a.pdf,Mr. Meyer discusses the economic outlook and t...,"Remarks by Mr. Laurence H. Meyer, a member of ...",1997-04-24 00:00:00,Mr. Meyer discusses the economic outlook and t...,Laurence H Meyer,1997-04-24,1997-04-24,771.179993
2,https://www.bis.org/review/r980102e.pdf,Mr. Kelley discusses the 'Millennium Bug' from...,"Remarks by Mr. Edward W. Kelley, Jr., a member...",1997-12-15 00:00:00,"Mr. Kelley discusses the ""Millennium Bug"" from...","Edward W Kelley, Jr",1997-12-15,1997-12-15,963.390015
3,https://www.bis.org/review/r971208d.pdf,Mr. Greenspan's remarks to the Economic Club o...,Remarks by the Chairman of the Board of the U ...,1997-12-02 00:00:00,Mr. Greenspan's remarks to the Economic Club o...,Alan Greenspan,1997-12-02,1997-12-02,971.679993
4,https://www.bis.org/review/r971202b.pdf,Mr. McDonough gives a US perspective on Econom...,Remarks by the President of the Federal Reserv...,1997-11-17 00:00:00,Mr. McDonough gives a US perspective on Econom...,William J McDonough,1997-11-17,1997-11-17,946.200012


Оставили только необходимые столбцы

In [26]:
#убираем лишние столбцы
speeches_final = speeches_1996_2023_frs_merge.drop\
(['url', 'title', 'description', 'date', 'author', 'date2', 'date_datetime'], axis=True)
display(speeches_final.head(20))

,text,close
0,Ms. Rivlin discusses the prudential regulation...,745.760010
1,Mr. Meyer discusses the economic outlook and t...,771.179993
2,"Mr. Kelley discusses the ""Millennium Bug"" from...",963.390015
3,Mr. Greenspan's remarks to the Economic Club o...,971.679993
4,Mr. McDonough gives a US perspective on Econom...,946.200012
5,Mr. Greenspan's remarks at the Center for Fina...,927.510010
6,Mr. Greenspan's testimony before the House of ...,916.659973
7,Remarks by Ms. Phillips at the Asset/Liability...,940.760010
8,Ms. Phillips discusses trends and challenges i...,903.679993
9,Mr. Kelley's testimony to the US House of Repr...,940.760010


Обновили индексы

In [37]:
speeches_final = speeches_final.reset_index(drop=True)
display(speeches_final.head(20))
print()
print(speeches_final.shape)

,text,close
0,Ms. Rivlin discusses the prudential regulation...,745.760010
1,Mr. Meyer discusses the economic outlook and t...,771.179993
2,"Mr. Kelley discusses the ""Millennium Bug"" from...",963.390015
3,Mr. Greenspan's remarks to the Economic Club o...,971.679993
4,Mr. McDonough gives a US perspective on Econom...,946.200012
5,Mr. Greenspan's remarks at the Center for Fina...,927.510010
6,Mr. Greenspan's testimony before the House of ...,916.659973
7,Remarks by Ms. Phillips at the Asset/Liability...,940.760010
8,Ms. Phillips discusses trends and challenges i...,903.679993
9,Mr. Kelley's testimony to the US House of Repr...,940.760010



(2216, 2)


Текст на кирилице отсутствует

In [35]:
#нет ли кирилицы в данных
pattern = r'[а-яА-ЯёЁ]'
kirill_list = []
n = 0
for i in range(0, len(speeches_final)):
    if re.search(pattern, speeches_final.loc[i, 'text']):
        n += 1
        kirill_list.append(i)
print(n)
print(n / len(speeches_final))

0
0.0


Осуществили токенизацию и лемматизацию текста

In [36]:
%%time
#лемматизируем текст
def get_wordnet_pos(word):
    """Map POS tag to first character lemmatize() accepts"""
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}
    return tag_dict.get(tag, wordnet.NOUN)
# 1. Init Lemmatizer
lemmatizer = WordNetLemmatizer()
# 3. Lemmatize a Sentence with the appropriate POS tag
lemm_list = []
for i in range(0, len(speeches_final)):
    latin = re.sub(r'[^a-zA-Z ]', ' ', speeches_final.loc[i, 'text'])
    latin_lemm = [lemmatizer.lemmatize(w, get_wordnet_pos(w)) for w in nltk.word_tokenize(latin)]
    latin_lemm_join = ' '.join(latin_lemm)
    lemm_list.append(latin_lemm_join)

Wall time: 1h 5min 14s


In [38]:
#длина списка
print(len(lemm_list))

2216


Присоединили лемматизированный текст к основному df

In [39]:
#присоединяем Series к DataFrame
lemm_text = pd.Series(lemm_list, name='lemm_text')
speeches_final_lemm = pd.concat([speeches_final, lemm_text.to_frame()], axis=1)
display(speeches_final_lemm.head())

,text,close,lemm_text
0,Ms. Rivlin discusses the prudential regulation...,745.760010,Ms Rivlin discus the prudential regulation of ...
1,Mr. Meyer discusses the economic outlook and t...,771.179993,Mr Meyer discus the economic outlook and the c...
2,"Mr. Kelley discusses the ""Millennium Bug"" from...",963.390015,Mr Kelley discus the Millennium Bug from a pub...
3,Mr. Greenspan's remarks to the Economic Club o...,971.679993,Mr Greenspan s remark to the Economic Club of ...
4,Mr. McDonough gives a US perspective on Econom...,946.200012,Mr McDonough give a US perspective on Economic...


In [51]:
print(speeches_final_lemm.dtypes)

text          object
close        float64
lemm_text     object
dtype: object


Добавили поле со значением Индекса S&P500 в формате int

In [53]:
speeches_final_lemm['close_int'] = speeches_final_lemm['close'].astype(int)
print(speeches_final_lemm.dtypes)
display(speeches_final_lemm.head())

text          object
close        float64
lemm_text     object
close_int      int32
dtype: object


,text,close,lemm_text,close_int
0,Ms. Rivlin discusses the prudential regulation...,745.760010,Ms Rivlin discus the prudential regulation of ...,745
1,Mr. Meyer discusses the economic outlook and t...,771.179993,Mr Meyer discus the economic outlook and the c...,771
2,"Mr. Kelley discusses the ""Millennium Bug"" from...",963.390015,Mr Kelley discus the Millennium Bug from a pub...,963
3,Mr. Greenspan's remarks to the Economic Club o...,971.679993,Mr Greenspan s remark to the Economic Club of ...,971
4,Mr. McDonough gives a US perspective on Econom...,946.200012,Mr McDonough give a US perspective on Economic...,946


Загрузили перечень стоп-слов английского языка для последующего использования

In [40]:
#англоязычные стоп-слова
stopwords = set(stopwords.words('english'))

### Работа с моделями

- Создали счетчик для TF-IDF, оценивающего важность каждого слова
- Разбили представленные данные на две выборки
- Посчитали TF-IDF для каждой выборки
- Обучили модели с использованием Линейной регрессии и Решающего дерева, оценили их качество
- По итогам сравнения качества моделей наилучший результат получили у модели, построенной с использованием Линейной регрессии, качество модели R2 составило 0.82
- Качество достаточно высокое - можно попробовать использовать данную модель для прогнозирования реакции рынков на выступления глав ЦБ

In [41]:
#создаем счетчик для TF-IDF
count_tf_idf = TfidfVectorizer(stop_words=stopwords)

In [54]:
#разбиваем данные на выборки
train, test = train_test_split(speeches_final_lemm, test_size=0.25, random_state=777)

In [55]:
#проверяем размеры выборок
print('train', train.shape)
print('test', test.shape)

train (1662, 4)
test (554, 4)


In [56]:
#считаем TF-IDF (оценка важности слова)
#train
tf_idf_train = count_tf_idf.fit_transform(train['lemm_text'])
#Если данные разделены на обучающую и тестовую выборки, функцию fit() запускайте только на обучающей
#test
tf_idf_test = count_tf_idf.transform(test['lemm_text'])

In [60]:
#LinearRegression
model_lr = LinearRegression()
model_lr.fit(tf_idf_train, train['close_int'])
predictions = model_lr.predict(tf_idf_test)
print("R2 =", r2_score(test['close_int'], predictions))

R2 = 0.8217413076001703


In [61]:
%%time
#дерево решений
for depth in range(1, 6):
    model_dt = DecisionTreeRegressor(random_state=12345, max_depth=depth)
    model_dt.fit(tf_idf_train, train['close_int'])
    predictions = model_dt.predict(tf_idf_test)
    print(depth)
    print("R2 =", r2_score(test['close_int'], predictions))
    print()

1
R2 = 0.49991967642254775

2
R2 = 0.544738973804904

3
R2 = 0.6126915997473803

4
R2 = 0.6302338652688134

5
R2 = 0.6201632726033104

Wall time: 1.93 s
